# Introducción a las tools ⚒️


En este laboratorio trabajaremos tres maneras de proporcionar herramientas a un LLM, desde la mas directa a la mas "abstraida":
1. Mediante un json schema
2. Mediante una clase pydantic
3. Mediante un servidor MCP
Doc para los primeros dos puntos: https://platform.openai.com/docs/guides/function-calling

Y trabajaremmos diferentes estrategias para consumir las tools
1. De manera atómica
2. De manera iterativa limitada
3. De manera iterativa limitada o hasta terminar la tarea
4. Interrumpiendo la ejecución para preguntar al usuario o a otro sistema cuya respuesta "puede tardar"



Siguiendo con la temática financiera, vamos a construir un bot capaz de aconsejarnos sobre acciones. Para ello, podemos usar alguno de los servicios que nos ofrecen datos en tiempo real de manera gratuita (con limitaciones)
Por ejemplo, https://finnhub.io/ nos proporciona hasta 60 llamadas por minuto en su versión gratuita.
Entramos y nos registramos para tener una api key

In [ ]:
%pip install finnhub-python

In [ ]:
import finnhub
from openai import AzureOpenAI
finnhub_api_key = ""
finnhub_client = finnhub.Client(api_key=finnhub_api_key)



Los sistemas bursatiles utilizan "etiquetas de cotización", el primer paso para que nuestro sistema pueda navegar por este tipo de datos podría ser darle la capacidad de convertir el nombre coloquial de un valor a su nombre técnico
finnhub tiene una funcion para esto: 


In [ ]:
finnhub_client.symbol_lookup('apple')

In [ ]:
import openai
import json
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_symbol",
            "description": "Obten el nombre tecnico de una accion, prerequisito para usar otras herramientas",
            "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "El nombre que ha dado el usuario a la accion o empresa"}}}}
        }
]
def llm_con_tools(input_text):
    client = AzureOpenAI(
        api_version="2025-03-01-preview",
        azure_endpoint="",
        api_key=""
    )
    systemprompt = f"""
    Eres un asistente de inversiones.
    Ayudas a un usuario a resolver sus dudas. Tienes herramientas para tener datos actualizados, usalas.
    """
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": systemprompt}, {"role": "user", "content": input_text}],
        tools=tools,
    )
    #Detectamos si se ha usado una herramienta
    if response.choices[0].finish_reason == "tool_calls":
        tool_call = response.choices[0].message.tool_calls[0]
        args = json.loads(tool_call.function.arguments)
        return args
    return response.choices[0].message.content

pregunta = input("¿En que te puedo ayudar?")
print(llm_con_tools(pregunta))

Parece que usa adecuadamente la herramienta. Vamos a crear otro metodo para ejecutar las tools

In [ ]:
import openai
import json
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_symbol",
            "description": "Obten el nombre tecnico de una accion, prerequisito para usar otras herramientas",
            "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "El nombre que ha dado el usuario"}}}}
        }
]
def run_tool(tool_name, args):
    print(f"Ejecutando herramienta: {tool_name}")
    print(f"Argumentos: {args}")
    match tool_name:
        case "get_stock_symbol":
            return finnhub_client.symbol_lookup(args["query"])
        case _:
            return "No se ha encontrado la herramienta"

def llm_con_tools(input_text):
    client = AzureOpenAI(
        api_version="2025-03-01-preview",
        azure_endpoint="",
        api_key=""
    )
    systemprompt = f"""
    Eres un asistente de inversiones.
    Ayudas a un usuario a resolver sus dudas. Tienes herramientas para tener datos actualizados, usalas.
    """
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": systemprompt}, {"role": "user", "content": input_text}],
        tools=tools
    )
    #Detectamos si se ha usado una herramienta
    if response.choices[0].finish_reason == "tool_calls":
        tool_call = response.choices[0].message.tool_calls[0]
        args = json.loads(tool_call.function.arguments)
        tool_name = tool_call.function.name
        tool_result = run_tool(tool_name, args)
        return tool_result
    return response.choices[0].message.content

pregunta = input("¿En que te puedo ayudar?")
print(llm_con_tools(pregunta))

Vamos a darle mas capacidades, creemos otra herramienta para sacar datos básicos de un valor.

Vamos a hacerlo con pydantic.


In [ ]:
print(finnhub_client.company_basic_financials('MSFT', 'all'))

In [ ]:
import pydantic
from pydantic import Field, BaseModel
from openai import pydantic_function_tool
import json

#Importante describir tanto la tool en si como cada uno de los campos
class BasicFinancials(BaseModel):
    """
    Obtiene los datos basicos de una accion o empresa a partir de su simbolo o nombre técnico (stock ticker symbol)
    """
    symbol: str = Field(description="El nombre tecnico de la accion de la empresa a consultar (stock ticker symbol), por ejemplo AAPL o MSFT")
    #metric: str = Field(description="El tipo de dato que se desea obtener,usa solo all")



tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_symbol",
            "description": "Obten el nombre tecnico (stock ticker symbol) de una accion",
            "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "El nombre que ha dado el usuario"}}}}
        }
]

tools.append(pydantic_function_tool(BasicFinancials))

In [ ]:
tools

In [ ]:




def run_tool(tool_name, args):
    print(f"Ejecutando herramienta: {tool_name}")
    print(f"Argumentos: {args}")
    match tool_name:
        case "get_stock_symbol":
            return finnhub_client.symbol_lookup(args["query"])
        case "BasicFinancials":
            return finnhub_client.company_basic_financials(args["symbol"], "all")
        case _:
            return "No se ha encontrado la herramienta"

def llm_con_tools(input_text):
    client = AzureOpenAI(     api_version="2025-03-01-preview",     azure_endpoint="",     api_key="" )
    systemprompt = f"""
    Eres un asistente de inversiones.
    Ayudas a un usuario a resolver sus dudas. Tienes herramientas para tener datos actualizados, usalas.
    """
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": systemprompt}, {"role": "user", "content": input_text}],
        tools=tools
    )
    #Detectamos si se ha usado una herramienta
    if response.choices[0].finish_reason == "tool_calls":
        print(response.choices[0].message.tool_calls)
        tool_call = response.choices[0].message.tool_calls[0]
        args = json.loads(tool_call.function.arguments)
        tool_name = tool_call.function.name
        tool_result = run_tool(tool_name, args)
        return tool_result
    return response.choices[0].message.content

pregunta = input("¿En que te puedo ayudar?")
print(llm_con_tools(pregunta))

Para que pueda usar varias herramientas, necesitamos que pueda ejecutarse varias veces. Vamos a modificarlo.

In [ ]:

def run_tool(tool_name, args):
    print(f"Ejecutando herramienta: {tool_name}")
    print(f"Argumentos: {args}")
    match tool_name:
        case "get_stock_symbol":
            return finnhub_client.symbol_lookup(args["query"])
        case "BasicFinancials":
            return finnhub_client.company_basic_financials(args["symbol"], "all")
        case _:
            return "No se ha encontrado la herramienta"

def llm_con_tools_multi_turno(input_text):
    numero_de_turnos = 3
    client = AzureOpenAI(
        api_version="2025-03-01-preview",
        azure_endpoint="",
        api_key=""
    )
    systemprompt = f"""
    Eres un asistente de inversiones.
    Ayudas a un usuario a resolver sus dudas. Tienes herramientas para tener datos actualizados, usalas. Usa solo una llamada en cada turno.
    """  
    messages=[{"role": "system", "content": systemprompt}, {"role": "user", "content": input_text}]
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        tools=tools
    )

    #Detectamos si se ha usado una herramienta
    if response.choices[0].finish_reason == "tool_calls":
        messages.append({"role": "assistant", "tool_calls": response.choices[0].message.tool_calls})
        tool_call = response.choices[0].message.tool_calls[0]
        args = json.loads(tool_call.function.arguments)
        tool_name = tool_call.function.name
        tool_result = run_tool(tool_name, args)
        messages.append({"role": "tool", 
                         "tool_call_id": tool_call.id,
                         "content": str(tool_result)})
    for turno in range(numero_de_turnos):
        print(f"Turno {turno}")
        print(messages)
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages,
            tools=tools
        )
        print (response)
        if response.choices[0].finish_reason == "tool_calls":
            print("Se ha usado una herramienta", response.choices[0].message.tool_calls )
            messages.append({"role": "assistant", "tool_calls": response.choices[0].message.tool_calls})
            tool_call = response.choices[0].message.tool_calls[0]
            args = json.loads(tool_call.function.arguments)
            tool_name = tool_call.function.name
            tool_result = run_tool(tool_name, args)
            messages.append({"role": "tool", 
                            "tool_call_id": tool_call.id,
                            "content": str(tool_result)})
        else:
            print("No se ha usado una herramienta")
            return response.choices[0].message.content
        print(messages)
    return response.choices[0].message.content

pregunta = input("¿En que te puedo ayudar?")
print(llm_con_tools_multi_turno(pregunta))

Vamos a añadir observabilidad

In [ ]:
import logfire
logfire.configure(token="")
logfire.instrument_openai() 


Y a permitir que use mas de una tool por turno

In [ ]:

def run_tool(tool_name, args):
    print(f"Ejecutando herramienta: {tool_name}")
    print(f"Argumentos: {args}")
    match tool_name:
        case "get_stock_symbol":
            return finnhub_client.symbol_lookup(args["query"])
        case "BasicFinancials":
            return finnhub_client.company_basic_financials(args["symbol"], "all")
        case _:
            return "No se ha encontrado la herramienta"

def llm_con_tools_multi_turno(input_text):
    numero_de_turnos = 4
    client = AzureOpenAI(     api_version="2025-03-01-preview",     azure_endpoint="",     api_key="" )
    systemprompt = f"""
    Eres un asistente de inversiones.
    Ayudas a un usuario a resolver sus dudas. Tienes herramientas para tener datos actualizados, usalas. 
    """  
    messages=[{"role": "system", "content": systemprompt}, {"role": "user", "content": input_text}]
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        tools=tools
    )

    # Detectamos si se ha usado una herramienta
    if response.choices[0].finish_reason == "tool_calls":
        for tool_call in response.choices[0].message.tool_calls:
            messages.append({"role": "assistant", "tool_calls": [tool_call]})
            args = json.loads(tool_call.function.arguments)
            tool_name = tool_call.function.name
            tool_result = run_tool(tool_name, args)
            messages.append({"role": "tool", 
                             "tool_call_id": tool_call.id,
                             "content": str(tool_result)})
    for turno in range(numero_de_turnos):
        print(f"Turno {turno}")
        print(messages)
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages,
            tools=tools
        )
        print(response)
        if response.choices[0].finish_reason == "tool_calls":
            print("Se ha usado una herramienta", response.choices[0].message.tool_calls)
            for tool_call in response.choices[0].message.tool_calls:
                messages.append({"role": "assistant", "tool_calls": [tool_call]})
                args = json.loads(tool_call.function.arguments)
                tool_name = tool_call.function.name
                tool_result = run_tool(tool_name, args)
                messages.append({"role": "tool", 
                                 "tool_call_id": tool_call.id,
                                 "content": str(tool_result)})
        else:
            print("No se ha usado una herramienta")
            return response.choices[0].message.content
        print(messages)
    return response.choices[0].message.content

pregunta = input("¿En que te puedo ayudar?")
print(llm_con_tools_multi_turno(pregunta))

Tratemos de hacer el sistema mas eficiente

In [ ]:
import pydantic
from pydantic import Field, BaseModel
from openai import pydantic_function_tool
import json

#Importante describir tanto la tool en si como cada uno de los campos
class BasicFinancials(BaseModel):
    """
    Obtiene los datos basicos de una accion
    """
    symbol: str = Field(description="El nombre tecnico de la accion")
    metric: str = Field(description="El tipo de dato que se desea obtener", enum=[
        "10DayAverageTradingVolume",
        "13WeekPriceReturnDaily",
        "26WeekPriceReturnDaily",
        "3MonthADReturnStd",
        "3MonthAverageTradingVolume",
        "52WeekHigh",
        "52WeekHighDate",
        "52WeekLow",
        "52WeekLowDate",
        "52WeekPriceReturnDaily",
        "5DayPriceReturnDaily",
        "assetTurnoverAnnual",
        "assetTurnoverTTM",
        "beta",
        "bookValuePerShareAnnual",
        "bookValuePerShareQuarterly",
        "bookValueShareGrowth5Y",
        "capexCagr5Y",
        "cashFlowPerShareAnnual",
        "cashFlowPerShareQuarterly",
        "cashFlowPerShareTTM",
        "cashPerSharePerShareAnnual",
        "cashPerSharePerShareQuarterly",
        "currentDividendYieldTTM",
        "currentEv/freeCashFlowAnnual",
        "currentEv/freeCashFlowTTM",
        "currentRatioAnnual",
        "currentRatioQuarterly",
        "dividendGrowthRate5Y",
        "dividendPerShareAnnual",
        "dividendPerShareTTM",
        "dividendYieldIndicatedAnnual",
        "ebitdPerShareAnnual",
        "ebitdPerShareTTM",
        "ebitdaCagr5Y",
        "ebitdaInterimCagr5Y",
        "enterpriseValue",
        "epsAnnual",
        "epsBasicExclExtraItemsAnnual",
        "epsBasicExclExtraItemsTTM",
        "epsExclExtraItemsAnnual",
        "epsExclExtraItemsTTM",
        "epsGrowth3Y",
        "epsGrowth5Y",
        "epsGrowthQuarterlyYoy",
        "epsGrowthTTMYoy",
        "epsInclExtraItemsAnnual",
        "epsInclExtraItemsTTM",
        "epsNormalizedAnnual",
        "epsTTM",
        "focfCagr5Y",
        "forwardPE",
        "grossMargin5Y",
        "grossMarginAnnual",
        "grossMarginTTM",
        "inventoryTurnoverAnnual",
        "inventoryTurnoverTTM",
        "longTermDebt/equityAnnual",
        "longTermDebt/equityQuarterly",
        "marketCapitalization",
        "monthToDatePriceReturnDaily",
        "netIncomeEmployeeAnnual",
        "netIncomeEmployeeTTM",
        "netInterestCoverageAnnual",
        "netInterestCoverageTTM",
        "netMarginGrowth5Y",
        "netProfitMargin5Y",
        "netProfitMarginAnnual",
        "netProfitMarginTTM",
        "operatingMargin5Y",
        "operatingMarginAnnual",
        "operatingMarginTTM",
        "payoutRatioAnnual",
        "payoutRatioTTM",
        "pb",
        "pbAnnual",
        "pbQuarterly",
        "pcfShareAnnual",
        "pcfShareTTM",
        "peAnnual",
        "peBasicExclExtraTTM",
        "peExclExtraAnnual",
        "peExclExtraTTM",
        "peInclExtraTTM",
        "peNormalizedAnnual",
        "peTTM",
        "pegTTM",
        "pfcfShareAnnual",
        "pfcfShareTTM",
        "pretaxMargin5Y",
        "pretaxMarginAnnual",
        "pretaxMarginTTM",
        "priceRelativeToS&P50013Week",
        "priceRelativeToS&P50026Week",
        "priceRelativeToS&P5004Week",
        "priceRelativeToS&P50052Week",
        "priceRelativeToS&P500Ytd",
        "psAnnual",
        "psTTM",
        "ptbvAnnual",
        "ptbvQuarterly",
        "quickRatioAnnual",
        "quickRatioQuarterly",
        "receivablesTurnoverAnnual",
        "receivablesTurnoverTTM",
        "revenueEmployeeAnnual",
        "revenueEmployeeTTM",
        "revenueGrowth3Y",
        "revenueGrowth5Y",
        "revenueGrowthQuarterlyYoy",
        "revenueGrowthTTMYoy",
        "revenuePerShareAnnual",
        "revenuePerShareTTM",
        "revenueShareGrowth5Y",
        "roa5Y",
        "roaRfy",
        "roaTTM",
        "roe5Y",
        "roeRfy",
        "roeTTM",
        "roi5Y",
        "roiAnnual",
        "roiTTM",
        "tangibleBookValuePerShareAnnual",
        "tangibleBookValuePerShareQuarterly",
        "tbvCagr5Y",
        "totalDebt/totalEquityAnnual",
        "totalDebt/totalEquityQuarterly",
        "yearToDatePriceReturnDaily"
    ])



tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_symbol",
            "description": "Obten el nombre tecnico de una accion, prerequisito para usar otras herramientas",
            "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "El nombre que ha dado el usuario"}}}}
        }
]

tools.append(pydantic_function_tool(BasicFinancials))

A veces la api responde con todo a pesar de mandarle una métrica, debemos definir un filtrado adicional.

In [ ]:
def parse_financials(symbol, metric):
    all_data = finnhub_client.company_basic_financials(symbol, 'all')
    if "metric" in all_data and metric in all_data["metric"]:
        return all_data["metric"][metric]
    else:
        return "No se ha encontrado esa métrica"
def run_tool(tool_name, args):
    print(f"Ejecutando herramienta: {tool_name}")
    print(f"Argumentos: {args}")
    match tool_name:
        case "get_stock_symbol":
            return finnhub_client.symbol_lookup(args["query"])
        case "BasicFinancials":
            return parse_financials(args["symbol"], args["metric"])
        case _:
            return "No se ha encontrado la herramienta"

def llm_con_tools_multi_turno(input_text):
    numero_de_turnos = 4
    client = AzureOpenAI(
    api_version="2025-03-01-preview",
    azure_endpoint="",
    api_key=""
    )
    systemprompt = f"""
    Eres un asistente de inversiones.
    Ayudas a un usuario a resolver sus dudas. Tienes herramientas para tener datos actualizados, usalas. 
    """  
    messages=[{"role": "system", "content": systemprompt}, {"role": "user", "content": input_text}]
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        tools=tools,
        temperature=0.0,
        max_tokens=16384
    )

    # Detectamos si se ha usado una herramienta
    if response.choices[0].finish_reason == "tool_calls":
        for tool_call in response.choices[0].message.tool_calls:
            messages.append({"role": "assistant", "tool_calls": [tool_call]})
            args = json.loads(tool_call.function.arguments)
            tool_name = tool_call.function.name
            tool_result = run_tool(tool_name, args)
            messages.append({"role": "tool", 
                             "tool_call_id": tool_call.id,
                             "content": str(tool_result)})
    for turno in range(numero_de_turnos):
        print(f"Turno {turno}")
        print(messages)
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages,
            tools=tools
        )
        print(response)
        if response.choices[0].finish_reason == "tool_calls":
            print("Se ha usado una herramienta", response.choices[0].message.tool_calls)
            for tool_call in response.choices[0].message.tool_calls:
                messages.append({"role": "assistant", "tool_calls": [tool_call]})
                args = json.loads(tool_call.function.arguments)
                tool_name = tool_call.function.name
                tool_result = run_tool(tool_name, args)
                messages.append({"role": "tool", 
                                 "tool_call_id": tool_call.id,
                                 "content": str(tool_result)})
        else:
            print("No se ha usado una herramienta")
            return response.choices[0].message.content
        print(messages)
    return response.choices[0].message.content

pregunta = input("¿En que te puedo ayudar?")
print(llm_con_tools_multi_turno(pregunta))

Intentar que una IA (O un humano) dé consejos sin contexto, puede ser un fracaso -especialmente en temas tan complejos como este-. Vamos a darle una última herramienta para que pueda contextualizar mejor, también vamos a cambiar a un LRM (Large reasoning model), o modelo razonador, que permitirá que el modelo de respuestas con mas profundida.


In [ ]:
import pydantic
from pydantic import Field, BaseModel
from openai import pydantic_function_tool
import json

#Importante describir tanto la tool en si como cada uno de los campos
class BasicFinancials(BaseModel):
    """
    Obtiene los datos basicos de una accion
    """
    symbol: str = Field(description="El nombre tecnico de la accion")
    metric: str = Field(description="El tipo de dato que se desea obtener", enum=[
        "10DayAverageTradingVolume",
        "13WeekPriceReturnDaily",
        "26WeekPriceReturnDaily",
        "3MonthADReturnStd",
        "3MonthAverageTradingVolume",
        "52WeekHigh",
        "52WeekHighDate",
        "52WeekLow",
        "52WeekLowDate",
        "52WeekPriceReturnDaily",
        "5DayPriceReturnDaily",
        "assetTurnoverAnnual",
        "assetTurnoverTTM",
        "beta",
        "bookValuePerShareAnnual",
        "bookValuePerShareQuarterly",
        "bookValueShareGrowth5Y",
        "capexCagr5Y",
        "cashFlowPerShareAnnual",
        "cashFlowPerShareQuarterly",
        "cashFlowPerShareTTM",
        "cashPerSharePerShareAnnual",
        "cashPerSharePerShareQuarterly",
        "currentDividendYieldTTM",
        "currentEv/freeCashFlowAnnual",
        "currentEv/freeCashFlowTTM",
        "currentRatioAnnual",
        "currentRatioQuarterly",
        "dividendGrowthRate5Y",
        "dividendPerShareAnnual",
        "dividendPerShareTTM",
        "dividendYieldIndicatedAnnual",
        "ebitdPerShareAnnual",
        "ebitdPerShareTTM",
        "ebitdaCagr5Y",
        "ebitdaInterimCagr5Y",
        "enterpriseValue",
        "epsAnnual",
        "epsBasicExclExtraItemsAnnual",
        "epsBasicExclExtraItemsTTM",
        "epsExclExtraItemsAnnual",
        "epsExclExtraItemsTTM",
        "epsGrowth3Y",
        "epsGrowth5Y",
        "epsGrowthQuarterlyYoy",
        "epsGrowthTTMYoy",
        "epsInclExtraItemsAnnual",
        "epsInclExtraItemsTTM",
        "epsNormalizedAnnual",
        "epsTTM",
        "focfCagr5Y",
        "forwardPE",
        "grossMargin5Y",
        "grossMarginAnnual",
        "grossMarginTTM",
        "inventoryTurnoverAnnual",
        "inventoryTurnoverTTM",
        "longTermDebt/equityAnnual",
        "longTermDebt/equityQuarterly",
        "marketCapitalization",
        "monthToDatePriceReturnDaily",
        "netIncomeEmployeeAnnual",
        "netIncomeEmployeeTTM",
        "netInterestCoverageAnnual",
        "netInterestCoverageTTM",
        "netMarginGrowth5Y",
        "netProfitMargin5Y",
        "netProfitMarginAnnual",
        "netProfitMarginTTM",
        "operatingMargin5Y",
        "operatingMarginAnnual",
        "operatingMarginTTM",
        "payoutRatioAnnual",
        "payoutRatioTTM",
        "pb",
        "pbAnnual",
        "pbQuarterly",
        "pcfShareAnnual",
        "pcfShareTTM",
        "peAnnual",
        "peBasicExclExtraTTM",
        "peExclExtraAnnual",
        "peExclExtraTTM",
        "peInclExtraTTM",
        "peNormalizedAnnual",
        "peTTM",
        "pegTTM",
        "pfcfShareAnnual",
        "pfcfShareTTM",
        "pretaxMargin5Y",
        "pretaxMarginAnnual",
        "pretaxMarginTTM",
        "priceRelativeToS&P50013Week",
        "priceRelativeToS&P50026Week",
        "priceRelativeToS&P5004Week",
        "priceRelativeToS&P50052Week",
        "priceRelativeToS&P500Ytd",
        "psAnnual",
        "psTTM",
        "ptbvAnnual",
        "ptbvQuarterly",
        "quickRatioAnnual",
        "quickRatioQuarterly",
        "receivablesTurnoverAnnual",
        "receivablesTurnoverTTM",
        "revenueEmployeeAnnual",
        "revenueEmployeeTTM",
        "revenueGrowth3Y",
        "revenueGrowth5Y",
        "revenueGrowthQuarterlyYoy",
        "revenueGrowthTTMYoy",
        "revenuePerShareAnnual",
        "revenuePerShareTTM",
        "revenueShareGrowth5Y",
        "roa5Y",
        "roaRfy",
        "roaTTM",
        "roe5Y",
        "roeRfy",
        "roeTTM",
        "roi5Y",
        "roiAnnual",
        "roiTTM",
        "tangibleBookValuePerShareAnnual",
        "tangibleBookValuePerShareQuarterly",
        "tbvCagr5Y",
        "totalDebt/totalEquityAnnual",
        "totalDebt/totalEquityQuarterly",
        "yearToDatePriceReturnDaily"
    ])



tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_symbol",
            "description": "Obten el nombre tecnico de una accion, prerequisito para usar otras herramientas",
            "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "El nombre que ha dado el usuario"}}}}
        }
]
class QueryUser(BaseModel):
    """
    Herramienta para consultar al usuario. Usala con frecuencia si no tienes suficiente contexto para dar una buena respuesta o si alguna de las herramientas no te da una respuesta clara.
    """
    question: str = Field(description="Pregunta que se le hace al usuario")

tools.append(pydantic_function_tool(QueryUser))
tools.append(pydantic_function_tool(BasicFinancials))

In [ ]:
def parse_financials(symbol, metric):
    all_data = finnhub_client.company_basic_financials(symbol, 'all')
    if "metric" in all_data and metric in all_data["metric"]:
        return all_data["metric"][metric]
    else:
        return "No se ha encontrado esa métrica"
def run_tool(tool_name, args):
    print(f"Ejecutando herramienta: {tool_name}")
    print(f"Argumentos: {args}")
    match tool_name:
        case "get_stock_symbol":
            return finnhub_client.symbol_lookup(args["query"])
        case "BasicFinancials":
            return parse_financials(args["symbol"], args["metric"])
        case "QueryUser":
            return input(args["question"])
        case _:
            return "No se ha encontrado la herramienta"

def llm_con_tools_multi_turno(input_text):
    numero_de_turnos = 10
    client = AzureOpenAI(     api_version="2025-03-01-preview",     azure_endpoint="",     api_key="" )
    systemprompt = f"""
    Eres un asistente de inversiones.
    Ayudas a un usuario a resolver sus dudas. Tienes herramientas para tener datos actualizados, usalas. 
    Si no tienes suficiente contexto para dar una buena respuesta o si alguna de las herramientas no te da una respuesta clara, usa la herramienta QueryUser. Es mejor usarla que dar una respuesta genérica.
    Tambien puedes usarla para continuar la conversacion si crees que hay algo interesante que decirle al usuario.
    """  
    messages=[{"role": "system", "content": systemprompt}, {"role": "user", "content": input_text}]
    response = client.chat.completions.create(
        model="o4-mini",
        messages=messages,
        reasoning_effort="medium",
        tools=tools
    )

    # Detectamos si se ha usado una herramienta
    if response.choices[0].finish_reason == "tool_calls":
        for tool_call in response.choices[0].message.tool_calls:
            messages.append({"role": "assistant", "tool_calls": [tool_call]})
            args = json.loads(tool_call.function.arguments)
            tool_name = tool_call.function.name
            tool_result = run_tool(tool_name, args)
            messages.append({"role": "tool", 
                             "tool_call_id": tool_call.id,
                             "content": str(tool_result)})
    for turno in range(numero_de_turnos):
        print(f"Turno {turno}")
        print(messages)
        response = client.chat.completions.create(
            model="o4-mini",
            messages=messages,
            reasoning_effort="medium",
            tools=tools
        )
        print(response)
        if response.choices[0].finish_reason == "tool_calls":
            print("Se ha usado una herramienta", response.choices[0].message.tool_calls)
            for tool_call in response.choices[0].message.tool_calls:
                messages.append({"role": "assistant", "tool_calls": [tool_call]})
                args = json.loads(tool_call.function.arguments)
                tool_name = tool_call.function.name
                tool_result = run_tool(tool_name, args)
                messages.append({"role": "tool", 
                                 "tool_call_id": tool_call.id,
                                 "content": str(tool_result)})
        else:
            print("No se ha usado una herramienta")
            return response.choices[0].message.content
        print(messages)
    return response.choices[0].message.content

pregunta = input("¿En que te puedo ayudar?")
print(llm_con_tools_multi_turno(pregunta))

Detalles para hacer foco:
- Coherencia en los prompts y en los nombres de las herramientas
- Podemos sesgar al modelo para que use mas o menos una herramienta
- Cambia mucho el sistema dependiendo del modelo. Mas todavía si pasamos de LLM a LRM
- Los modelos LRM son menos deterministas

## Adentremonos en el mundo del MCP

MCP permite en su versión actual y en su uso básico, exponer a la IA tres tipos de elementos:
- Prompts (Instrucciones preconfiguradas para tareas concretas)
- Resources (Acceso a archivos de cualquier tipo)
- Tools (Uso de herramientas)

Para cada uno de estos elementos, existe un /list que permite que la IA haga "autodescubrimiento" al empezar a usar el MCP, lo que le da mucha flexibilidad (e incrementa los costes y el contexto de manera variable)
Vamos a comenzar con las tools. Ya que el resto de elementos **no son soportados aun por openAI**

Usaremos fastMCP sobre http y lo haremos sobre .py para organizarlo mejor. Para poder usar este server debes usar un tunel de algún tipo, te recomiendo devtunnel.

In [ ]:
%pip install fastmcp 
#no te olvides de instalar la dependencia.

Instalamos y configuramos devtunnel: https://learn.microsoft.com/en-us/azure/developer/dev-tunnels/get-started?tabs=windows

Ejecutamos el servidor con python financial_data_mcp.py

Vamos a configurar nuestro cliente de openai para usar el server en vez de las tools configuradas anteriormente

Doc: https://platform.openai.com/docs/guides/tools-remote-mcp

In [ ]:
mcp_url = "https://l3dlqc2h-8002.uks1.devtunnels.ms"
#no te olvides de añadir /mcp/ al final si usas fastmcp
mcp_url = mcp_url + "/mcp/"
mcp_tool = {
    "type": "mcp",
    "server_label":"Datos_financieros_actualizados",
    "server_url": mcp_url,
    "require_approval":"never"
}
tools = [mcp_tool]

Nota: Aqui he migrado a responses API SDK porque completions no tiene la capacidad de MCP. Es bueno que conozcas ambas, y que para proyectos nuevos uses Responses.

Nota 2: Fijaos como se simplifica el código

In [ ]:
import logfire
logfire.configure(token="")
logfire.instrument_openai() 

In [ ]:
import json
from openai import AzureOpenAI




def llm_con_mcp(input_text):
    client = AzureOpenAI(     api_version="2025-03-01-preview",     azure_endpoint="",     api_key="" )
    systemprompt = (
        """
        Eres un asistente de inversiones.
        Ayudas a un usuario a resolver sus dudas. Tienes herramientas para tener datos actualizados, usalas.
        """
    )

    # Primera creación de respuesta con Responses API
    response = client.responses.create(
        model="o4-mini",
        input=input_text,
        instructions=systemprompt,
        reasoning={"effort": "medium"},
        tools=tools,
        store=True
    )
    return (response.output_text)



pregunta = input("¿En que te puedo ayudar?")
print(llm_con_mcp(pregunta))

19:46:05.404 Responses API with 'o4-mini' [LLM]
A continuación, una comparativa muy a alto nivel de las dos compañías con datos TTM (últimos doce meses) y cifras aproximadas:

1. Tamaño (Market Cap)  
   • Apple (AAPL): ≈ 3,40 billones USD  
   • Microsoft (MSFT): ≈ 3,88 billones USD  

2. Valoración (P/E TTM)  
   • AAPL: 34,3×  
   • MSFT: 38,1×  

3. Rentabilidad por dividendo (yield TTM)  
   • AAPL: 0,45 %  
   • MSFT: 0,62 %  

4. Crecimiento de ingresos YoY (TTM)  
   • AAPL: +6,0 %  
   • MSFT: +14,9 %  

5. Rentabilidad sobre el patrimonio (ROE TTM)  
   • AAPL: 155 %  
   • MSFT: 32 %  

Principales conclusiones a muy alto nivel:  
- Microsoft es hoy ligeramente mayor por capitalización y cotiza a un P/E algo superior, reflejo de expectativas de crecimiento más elevadas (ingresos +15 % vs. +6 %).  
- Apple, por su parte, presume de una extraordinaria eficiencia de capital (ROE >150 %, potenciado por un balance muy optimizado), aunque su ritmo de expansión orgánica de ventas e

Es relativamente facil crear un sistema asi pero, aunque el modelo haya hecho varias llamadas y funcionado bien, no sabemos nada de lo que ha pasado. Vamos a tratar de mejorar la observabilidad.

In [ ]:
import json
import openai


def llm_con_mcp(input_text):
    client = AzureOpenAI(     api_version="2025-03-01-preview",     azure_endpoint="",     api_key="" )
    systemprompt = (
        """
        Eres un asistente de inversiones.
        Ayudas a un usuario a resolver sus dudas. Tienes herramientas para tener datos actualizados, usalas.
        """
    )

    # Primera creación de respuesta con Responses API
    response = client.responses.create(
        model="o4-mini",
        input=input_text,
        instructions=systemprompt,
        reasoning={"effort": "medium"},
        tools=tools,
        store=True
    )

    return (response.output_text)



pregunta = input("¿En que te puedo ayudar?")
print(llm_con_mcp(pregunta))

OpenAI y Claude permiten, en su versión de pago (en la PRO en openai), añadir MCP a sus modelos de chat. 

Antes de pasar al problema...
Un uso típico de los mcps es para aumentar las capacidades de los editores de codigo con IA ¿Lo conoces?